In [4]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from sklearn.cluster import KMeans

class TrainBookingSystem:
    def __init__(self, booked_seats_data):
        self.total_rows = 17
        self.total_cols = 6
        self.seats = [['O' for _ in range(1, self.total_cols + 1)] for _ in range(1, self.total_rows + 1)]
        self.booked_seats = {}
        self.base_seat_price = 800  
        booked_seats_data['Age Group Encoded'] = pd.Categorical(booked_seats_data['Preferred Age Group']).codes

        for index, row in booked_seats_data.iterrows():
            if row['Seat Booked'] == 1:
                seat_number = row['Seat Number']
                age_group = row['Preferred Age Group']
                self.booked_seats[seat_number] = age_group
                self.seats[row['Row'] - 1][row['Column'] - 1] = 'X'

        self.cluster_model = self.train_clustering_model(booked_seats_data)
        self.root = tk.Tk()
        self.root.title("Train Booking System")
        self.age_group = None
        self.recommendations = []

        self.create_age_selection()

    def train_clustering_model(self, booked_seats_data):
        X = booked_seats_data[['Row', 'Column', 'Age Group Encoded']]
        kmeans = KMeans(n_clusters=5, random_state=42)
        booked_seats_data['Cluster'] = kmeans.fit_predict(X)
        self.booked_seats_data = booked_seats_data
        return kmeans

    def create_age_selection(self):
        self.age_selection_frame = tk.Frame(self.root)
        self.age_selection_frame.pack(pady=20)

        tk.Label(self.age_selection_frame, text="Select Age Group:").pack(side=tk.LEFT)
        self.age_group_var = tk.StringVar(value='18-25')
        age_groups = ['18-25', '25-40', '40-60', '60+']
        for age_group in age_groups:
            tk.Radiobutton(self.age_selection_frame, text=age_group, variable=self.age_group_var, value=age_group).pack(side=tk.LEFT)

        tk.Button(self.age_selection_frame, text="Confirm Age Group", command=self.confirm_age_group).pack(side=tk.LEFT)

    def confirm_age_group(self):
        self.age_group = self.age_group_var.get()
        self.recommendations = self.get_recommended_seats(self.age_group)
        self.show_recommendations()

    def show_recommendations(self):
        self.age_selection_frame.pack_forget()
        self.recommendation_frame = tk.Frame(self.root)
        self.recommendation_frame.pack(pady=20)

        tk.Label(self.recommendation_frame, text=f"Recommended Seats for {self.age_group}:").pack()

        recommended_seats_str = ', '.join(map(str, self.recommendations))
        tk.Label(self.recommendation_frame, text=f"Seats: {recommended_seats_str}").pack()
        tk.Button(self.recommendation_frame, text="Proceed to Seat Selection", command=self.create_seat_grid).pack(pady=10)

    def create_seat_grid(self):
        self.recommendation_frame.pack_forget()
        self.grid_frame = tk.Frame(self.root)
        self.grid_frame.pack(pady=20)

        self.buttons = []
        for r in range(1, self.total_rows + 1):
            row_buttons = []
            for c in range(1, self.total_cols + 1):
                seat_number = (r - 1) * self.total_cols + c
                if seat_number in self.booked_seats:
                    color = 'red'  
                elif seat_number in self.recommendations:
                    color = 'blue'  
                else:
                    color = 'green'  
                
                button = tk.Button(self.grid_frame, text=seat_number, width=5, height=2, bg=color,
                                   command=lambda sn=seat_number: self.select_seat(sn))
                button.grid(row=r-1, column=c-1)
                row_buttons.append(button)
            self.buttons.append(row_buttons)

    def get_recommended_seats(self, age_group):
        recommended = []

        age_groups = ['18-25', '25-40', '40-60', '60+']
        age_group_label = pd.Categorical([age_group], categories=age_groups).codes[0]

        features_list = []
        for r in range(1, self.total_rows + 1):
            for c in range(1, self.total_cols + 1):
                seat_number = (r - 1) * self.total_cols + c
                if seat_number not in self.booked_seats:
                    if self.is_adjacent_seat_available(r, c, age_group):
                        features_list.append({
                            'Row': r,
                            'Column': c,
                            'Age Group Encoded': age_group_label
                        })

        features = pd.DataFrame(features_list)
        if not features.empty:
            clusters = self.cluster_model.predict(features)
            for i, row in features.iterrows():
                seat_number = (row['Row'] - 1) * self.total_cols + row['Column']
                cluster_label = clusters[i]
                if self.is_adjacent_seat_in_cluster(row['Row'], row['Column'], cluster_label):
                    recommended.append(seat_number)

        return recommended

    def is_adjacent_seat_available(self, row, col, age_group):
        adjacent_seats = [
            (row, col - 1), (row, col + 1)
        ]
        for r, c in adjacent_seats:
            if 1 <= r <= self.total_rows and 1 <= c <= self.total_cols:
                seat_number = (r - 1) * self.total_cols + c
                if seat_number in self.booked_seats and self.booked_seats[seat_number] == age_group:
                    return True
        return False

    def is_adjacent_seat_in_cluster(self, row, col, cluster_label):
        adjacent_seats = [
            (row, col - 1), (row, col + 1)
        ]
        for r, c in adjacent_seats:
            if 1 <= r <= self.total_rows and 1 <= c <= self.total_cols:
                seat_number = (r - 1) * self.total_cols + c
                features = pd.DataFrame([{
                    'Row': r,
                    'Column': c,
                    'Age Group Encoded': pd.Categorical([self.age_group], categories=['18-25', '25-40', '40-60', '60+']).codes[0]
                }])
                cluster = self.cluster_model.predict(features)[0]
                if cluster == cluster_label:
                    return True
        return False

    def select_seat(self, seat_number):
        row = (seat_number - 1) // self.total_cols + 1
        col = (seat_number - 1) % self.total_cols + 1

        if seat_number in self.booked_seats:
            messagebox.showwarning("Seat Already Booked", f"Seat {seat_number} is already booked.")
        else:
            self.book_seat(seat_number)

    def get_dynamic_price(self):
        remaining_seats_count = (self.total_rows * self.total_cols) - len(self.booked_seats)
        total_seats = self.total_rows * self.total_cols

        if remaining_seats_count > total_seats * 0.5:
            multiplier = 1.0  
        elif remaining_seats_count > total_seats * 0.2:
            multiplier = 1.2  
        else:
            multiplier = 1.5  

        return self.base_seat_price * multiplier

    def book_seat(self, seat_number):
        row = (seat_number - 1) // self.total_cols + 1
        col = (seat_number - 1) % self.total_cols + 1
        price = self.get_dynamic_price()

        self.booked_seats[seat_number] = self.age_group
        self.seats[row - 1][col - 1] = 'X'
        
        self.buttons[row - 1][col - 1].config(bg='red')
        
        messagebox.showinfo("Booking.......", f"Seat {seat_number} booking is in progress at Price: ₹{price:.2f}")

        self.buttons[row - 1][col - 1].config(state=tk.DISABLED)

        self.reset_to_age_selection()

    def reset_to_age_selection(self):
        self.grid_frame.pack_forget()
        self.create_age_selection()

    def run(self):
        self.root.mainloop()

# Load the booking data
booking_data = pd.read_csv('train_booking_data_.csv')  
booking_system = TrainBookingSystem(booking_data)
booking_system.run()
